# Imports

In [1]:
#Torch imports
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

#Misc imports
import os
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

# Functions

In [2]:
#Function to convert string to one hot encoding
def to_onehot(data):
    ascii_list = []
    elements_per_song = []
    for songs in data:
        elements_per_song.append(len(songs))
        for chars in songs:
            ascii_list.append(ord(chars))
    onehot_vals = np.eye(128)[ascii_list]
    return onehot_vals, np.array(ascii_list), np.array(elements_per_song)

In [3]:
#data is a list of list of chars, each string list is a song
#onehot_ascii output are shifted up by 1, need to figure out what to do with 0 array labels
def onehot_songs(data, sequence_length):
    max_num_chars = 0
    temp_list = []
    onehot_ascii = []
    onehot_songs = []
    
    #List of songs in ascii format
    for song in data:
        if len(song)>max_num_chars:
            max_num_chars = len(song) #Finds longest song length
        for chars in song:
            temp_list.append(ord(chars))
        onehot_ascii.append(temp_list)
        temp_list = []
        
    #To make divisible by sequence_length per batch
    max_num_chars = sequence_length-(max_num_chars%sequence_length)+max_num_chars
    codding = np.append(np.zeros((1,128)),np.eye(128),0)    
    
    for i, ascii_song in enumerate(onehot_ascii):
        ascii_song = np.array(ascii_song)+1 #since making first row of eye matrix all 0s
        needed_0s = max_num_chars-len(ascii_song)
        onehot_ascii[i] = np.pad(ascii_song,(0,needed_0s),'constant',constant_values=0)
        onehot_songs.append(codding[onehot_ascii[i]])
        
    return onehot_ascii, onehot_songs

In [4]:
#data is onehot array of every char in train
#labels are every ascii value of every char in train
def make_sequence(data,labels,sequence_length):
    num_groups = np.floor(len(labels)/sequence_length) #length labels same as data
    remainder = len(labels)%sequence_length
    #Add 1 here to grab one time step ahead for predicting labels
    label_groups = np.split(labels[0+1:len(labels)-remainder+1],num_groups)
    onehot_groups = np.vsplit(data[0:len(labels)-remainder],num_groups)
    return np.array(onehot_groups),np.array(label_groups)

In [5]:
#batch_size is the number of sequences per batch
def batching(onehot_data,labels,batch_size):
    num_batches = np.floor(len(labels)/batch_size)
    num_subtract = len(labels)%batch_size
    label_batches = np.vsplit(labels[0:len(labels)-num_subtract],num_batches)
    onehot_batches = np.split(onehot_data[0:len(labels)-num_subtract],num_batches,0)
    return np.array(onehot_batches),np.array(label_batches)

# RNN Class LSTM

In [24]:
class lstm_rnn(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers=1,batch_size=1,
                 sequence_length=25,batch_first = True):
        super(lstm_rnn, self).__init__()
        #Constants
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        
        #Initializing model
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first = batch_first)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self,data,initial):
        #data = data.view(self.batch_size, self.sequence_length, self.input_size)
        output,hidden = self.lstm(data,initial)
        out = self.fc(output.contiguous().view(-1,output.size(2)))
        print(out.shape)
        out = F.softmax(output)
        return out,hidden
        
    def init_hidden(self,zero=1):
        if zero == 0:
            #Random initialization
            initial_hidden = autograd.Variable(torch.randn(self.num_layers,self.batch_size,self.hidden_size))
        else:
            #Zero initialization
            initial_hidden = autograd.Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_size))
        return initial_hidden
    
    def init_cell(self,zero=1):
        if zero == 0:
            #Random initialization
            initial_cell = autograd.Variable(torch.randn(self.num_layers,self.batch_size,self.hidden_size))
        else:
            #Zero initialization
            initial_cell = autograd.Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_size))
        return initial_cell
    

# Data Formatting

In [7]:
file = open("./Data/input.txt")
text = file.readlines()
file.close()

text_array = np.asarray(text)

#Creates list of each song
indicies = np.where(text_array == '<start>\n') #Location of where each abc file starts
data = []
for i in range(len(indicies[0])):
    if i+1 == len(indicies[0]):
        #For the last abc file
        abc = text_array[indicies[0][i]:]
    else:
        abc = text_array[indicies[0][i]:indicies[0][i+1]]
    data.append(''.join(abc))

#print(data[0])
#print(data[1123])

In [8]:
#80 - 20 split on data -> training and validation
#Constants
train_len = int(np.floor(len(data)*0.8))
validation_len = len(data) - train_len

np.random.seed(0)
#Each index references to a single song
indxs = np.asarray(range(len(data)))
np.random.shuffle(indxs)

train_data = (np.asarray(data))[indxs[0:train_len]]
validation_data = (np.asarray(data))[indxs[train_len:]]

# Initialize Data and LSTM constants

In [9]:
#LSTM constants
input_size = 128 #num_classes = 128
hidden_size = 128
num_layers = 1
batch_first = True
batch_size = 500
sequence_length = 25

#Data
#combines all the songs and makes a single string, converted to ascii and onehot
onehot_train_full, train_labels_full, train_NumPerSong = to_onehot(train_data)
onehot_validation_full, validation_labels_full, validation_NumPerSong = to_onehot(validation_data)
#print(train_labels[0])
#print(onehot_train[0])

#or using onehot song encoddings
#onehot_train, train_labels_padded = onehot_songs(train_data,sequence_length)
#onehot_validation, validation_labels_padded = onehot_songs(validation_data,sequence_length)

#divide data based on sequence length
onehot_train, train_labels = make_sequence(onehot_train_full,
                                           train_labels_full,
                                           sequence_length)

onehot_validation, validation_labels = make_sequence(onehot_validation_full,
                                                     validation_labels_full,
                                                     sequence_length)

#Makes batches, labels are 1 time step ahead for predicting outputs
onehot_batches,batch_labels = batching(onehot_train,train_labels,batch_size)


# LSTM RNN

run first in single batches, each batch has 25 chars per(sequence length 25), just run it in order of the train data

In [14]:
lstm = lstm_rnn(input_size,hidden_size,num_layers,batch_size,
                sequence_length,batch_first = batch_first)
lstm.cuda()

print("Input feature size: ", lstm.input_size)
print("Number of hidden LSTM units: ", lstm.hidden_size)
print("Batch Size: ", lstm.batch_size)
print("Elements per sequence: ", lstm.sequence_length)
print("Number of LSTM layers: ", lstm.num_layers)
print("\n",lstm)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lstm.parameters(), lr=0.1)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

Input feature size:  128
Number of hidden LSTM units:  128
Batch Size:  500
Elements per sequence:  25
Number of LSTM layers:  1

 lstm_rnn(
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=128)
)


In [22]:
#Initialize hidden states
h_0 = lstm.init_hidden().cuda()
c_0 = lstm.init_cell().cuda()
states = (h_0,c_0)
loss = []
accuracy = []
num_batches = len(onehot_batches) #when using 500 per batch, results in 32 total batches
    
for epoch in range(5):
    for indx in range(num_batches):
        optimizer.zero_grad()
        in_data = autograd.Variable(torch.FloatTensor(onehot_batches[indx])).cuda()
        output,states = lstm(in_data,states)
        
        h_0 = autograd.Variable(states[0].data, requires_grad=True).cuda()
        c_0 = autograd.Variable(states[1].data, requires_grad=True).cuda()
        states = (h_0,c_0)
        
        #Labels are 1 time step ahead in time
        #500 by 25 per batch
        labels = batch_labels[indx].reshape(batch_size*sequence_length)
        labels = autograd.Variable(torch.LongTensor(labels)).cuda() 
                            
        #pretty sure that loss function will average over number of inputs
        #if not using linear layer
        #output = output.contiguous().view(-1,output.size(2))
        #if using linear layer, output should be shaped correctly already
        loss_temp = criterion(output,labels)
        loss.append(loss_temp.data)
        loss_temp.backward()
        optimizer.step()
        
        #vals,max_indxs = (output.data).max(2)
        #acc_temp = (max_indxs==(labels.data)).sum()
        #accuracy.append(acc_temp/sequence_length)
        #print(loss)
        #print(accuracy)
    print(epoch)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


ValueError: Expected 2 or 4 dimensions (got 3)

In [ ]:
plt.plot(loss)

In [ ]:
#write a function for this later 
primer = "<start>\n"
start_token = []
for chars in primer:
    start_token.append(ord(chars))
    
start_token = np.eye(128)[start_token]
start_token = torch.from_numpy(start_token).float()
start_token = autograd.Variable(start_token.view(1,8,input_size))
#print(start_token.argmax(1))

end = "<end>"
end_token = []
for chars in end:
    end_token.append(ord(chars))
    
end_token = np.eye(128)[end_token]

done = 1

#Generating music
h_0 = lstm.init_hidden()
c_0 = lstm.init_cell()
states = (h_0,c_0)
chars = []

#Each loop generates 1 char
for i in range(100):
    output,states = lstm(start_token,states)
    
    h_0 = autograd.Variable(states[0].data, requires_grad=True)
    c_0 = autograd.Variable(states[1].data, requires_grad=True)
    states = (h_0,c_0)
    
    #Currently I am taking the max of the hidden state as the letter that it
    #produces, the TA on piazza said that it is not ideal to take max,
    #can try implementing what he says later
    val,indx = states[0].data.max(2)
    chars.append(indx)
    
    start_token = torch.from_numpy(np.eye(128)[indx]).float()
    start_token = autograd.Variable(start_token.view(1,1,input_size))
    
    #Need to implement a check for "<end>", generation should stop once
    #"<end>" is produced

In [ ]:
#Converting back to strings
outputs = []
for letter in chars:
    outputs.append(chr(letter))

print(chars)
print(outputs)
